In [1]:
import argparse
import numpy as np
import pandas as pd
import itertools
import os
import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import roc_auc_score
import gc
from thefuzz import fuzz
from thefuzz import process
import time
import re
import recordlinkage
from sklearn.model_selection import train_test_split
from Levenshtein import ratio

gc.collect()

0

In [3]:
# This fuction is designed to find the id matched between wikidata and harvard index
def define_true_pairs(indexList1, indexList2, indexName1, indexName2):
    arrays = [indexList1, indexList2]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=[indexName1, indexName2])
    return index

In [2]:
'''
# this function will remove all special charaters -- including spaces
# but too slow comparing to replace -- used process_time() for evaluation
def clean_text(text): # fa
    a = ""
    text = a.join(char for char in text if char.isalnum())
    return text
'''
# Remove square blankets auto generated during data alignment process
def clean_text(text): # fb
    text = text.replace('[', '').replace(']','').replace("'", '')
    return text

def remove_spec_in_col(df, col):
    newCol = []
    for index, rowValue in df[col].iteritems():
        if pd.notnull(rowValue):
            newCol.append(clean_text(rowValue))
        else:
            newCol.append(np.nan)
    return newCol

In [13]:
# Define the function to split the label(full name) into first name and last name
def split_full_name(full_name):
    if pd.isnull(full_name) or full_name == '':
        return '', ''
    parts = full_name.split()
    first_name = ' '.join(parts[:-1])
    last_name = parts[-1]
    return first_name, last_name

# Define the function to convert each word in the first name to the desired format
def convert_to_initial(name):
    if pd.isnull(name) or name == '':
        return ''
    initials = [word[0].upper() + '.' for word in name.split()]
    return ' '.join(initials)

In [14]:
# Helper function to combine two multi index objects
def combine_multi_index(index1, index2):
    # Convert MultiIndex objects to DataFrames
    multi_index1 = index1.to_frame(index=False)
    multi_index2 = index2.to_frame(index=False)

    # Concatenate the DataFrames
    combined_index = pd.concat([multi_index1, multi_index2])

    # Remove duplicates
    combined_index = combined_index.drop_duplicates()

    # Convert back to MultiIndex
    combined_multi_index = pd.MultiIndex.from_frame(combined_index)
    
    return combined_multi_index

In [ ]:
def is_initial(name):
    parts = name.split()
    return all(len(part) == 2 and part[0].isalpha() and part[1] == '.' for part in parts)

def names_not_completely_different(name1, name2, threshold):
    return ratio(name1, name2) > threshold

In [ ]:
# Model to find the possible matching records between two datasets
def generate_match_pairs(dataset1, dataset2, lastName_threshold, firstName_threshold):  

    # Initialize indexer
    indexer = recordlinkage.Index()

    # Sorted neighborhood indexing on lastname
    indexer.sortedneighbourhood('lastName', window=3)
    
    # Blocking on author abbreviation - generate index pairs if 'exact match on author abbreviation'
    indexer.block(left_on = 'authorAbbrv', right_on='B & P Author Abbrev.')

    candidate_links = indexer.index(dataset1, dataset2)

    compare_cl = recordlinkage.Compare()

    # Exact match on author abbreviation, extra conditioning on the blocking
    compare_cl.exact('authorAbbrv', 'B & P Author Abbrev.', label='authorAbbrv')
    
    # Compare last names with Levenshtein distance
    # When threshold set to none, the feature table stores the similarity percentage calculated using Levenshtein distance
    compare_cl.string('lastName', 'lastName', method='damerau_levenshtein', threshold=None, label='lastName')

    # Compare first names with Levenshtein distance
    compare_cl.string('firstName', 'firstName', method='damerau_levenshtein', threshold=None, label='firstName')
    
    # Exact match on first name initial
    compare_cl.exact('firstName_initial', 'firstName_initial',label='firstName_initial')

    # Exact match on first name initial from label(normalised full name)
    compare_cl.exact('first_name_initial', 'first_name_initial', label='first_name_initial')

    # Exact match on date of birth
    compare_cl.exact('dateOfBirth', 'birthYear', label='dateOfBirth')

    # Exact match on date of death
    compare_cl.exact('dateOfDeath', 'deathYear', label='dateOfDeath')

    # Compute the comparison results
    features = compare_cl.compute(candidate_links, dataset1, dataset2)
    # print(len(features))
    
    # Filter pairs based on the given criteria
    index_pairs = []
    # Set the threshold of names_not_completely_different function (firstname check after initial comparison)
    threshold = 0.5

    for index, row in features.iterrows():
        a_index = index[0]
        b_index = index[1]
        
        # Store the record pairs if their author abbreviations are the same
        if row['authorAbbrv'] == 1:
            index_pairs.append(index)
            
        # Waterfall model to check criteria
        if row['lastName'] >= lastName_threshold:
            if row['firstName'] >= firstName_threshold:
                index_pairs.append(index)
            if row['firstName_initial'] == 1:
                firstName1 = dataset1.at[index[0], 'firstName']
                firstName2 = dataset2.at[index[1], 'firstName']
                if not (is_initial(firstName1) or is_initial(firstName2)) and names_not_completely_different(firstName1, firstName2, threshold):
                    index_pairs.append(index)
            if row['first_name_initial'] == 1:
                if not (is_initial(firstName1) or is_initial(firstName2)) and names_not_completely_different(firstName1, firstName2, threshold):
                    index_pairs.append(index)
            if row['firstName_initial'] == 1 and row['dateOfBirth'] == 1:
                index_pairs.append(index)
            if row['first_name_initial'] == 1 and row['dateOfBirth'] == 1:
                index_pairs.append(index)
            # if row['dateOfBirth'] == 1:
            #     index_pairs.append(index)
            if row['dateOfBirth'] == 1 and row['dateOfDeath'] == 1:
                index_pairs.append(index)
            # Check if b's label is in a's aliases list
            if pd.notna(dataset1.at[a_index, 'aliases']) and row['dateOfBirth'] == 1:
                aliases_list = [alias.strip() for alias in dataset1.at[a_index, 'aliases'].split(',')]
                for alias in aliases_list:
                    if pd.notna(dataset2.at[b_index, 'Standard/Label Name']) and pd.notna(alias) and ratio(dataset2.at[b_index, 'Standard/Label Name'], alias) >= 0.9:
                        index_pairs.append(index)
                        break              
            # Check if a's label is in b's aliases list
            if pd.notna(dataset2.at[b_index, 'Name']) and row['dateOfBirth'] == 1:
                aliases_list = [alias.strip() for alias in dataset2.at[b_index, 'Name'].split(',')]
                for alias in aliases_list:
                    if pd.notna(dataset1.at[a_index, 'label']) and pd.notna(alias) and ratio(dataset1.at[a_index, 'label'], alias) >= 0.9:
                        index_pairs.append(index)
                        break

    # Remove duplicate pairs
    index_pairs = list(set(index_pairs))
    
    # Convert index_pairs to a MultiIndex object
    multi_index = pd.MultiIndex.from_tuples(index_pairs, names=["wikiID", "harvardIndex"])

    return multi_index

In [ ]:
# Model to find the possible matching records between two datasets
def generate_match_pairs_bio(dataset1, dataset2, lastName_threshold, firstName_threshold):  

    # Initialize indexer
    indexer = recordlinkage.Index()

    # Sorted neighborhood indexing on lastname
    indexer.sortedneighbourhood('lastName', window=3)
    
    # Blocking on author abbreviation - generate index pairs if 'exact match on author abbreviation'
    # indexer.block(left_on = 'authorAbbrv', right_on='B & P Author Abbrev.')

    candidate_links = indexer.index(dataset1, dataset2)
    # print(len(candidate_links))

    compare_cl = recordlinkage.Compare()

    # Exact match on author abbreviation, extra conditioning on the blocking
    # compare_cl.exact('authorAbbrv', 'B & P Author Abbrev.', label='authorAbbrv')
    
    # Compare last names with Levenshtein distance
    # When threshold set to none, the feature table stores the similarity percentage calculated using Levenshtein distance
    compare_cl.string('lastName', 'lastName', method='damerau_levenshtein', threshold=None, label='lastName')

    # Compare first names with Levenshtein distance
    compare_cl.string('firstName', 'firstName', method='damerau_levenshtein', threshold=None, label='firstName')
    
    # Exact match on first name initial
    compare_cl.exact('firstName_initial', 'firstName_initial',label='firstName_initial')

    # Exact match on first name initial from label(normalised full name)
    compare_cl.exact('first_name_initial', 'first_name_initial', label='first_name_initial')

    # Exact match on date of birth
    compare_cl.exact('dateOfBirth', 'dateOfBirth', label='dateOfBirth')

    # Exact match on date of death
    compare_cl.exact('dateOfDeath', 'dateOfDeath', label='dateOfDeath')

    # Compute the comparison results
    features = compare_cl.compute(candidate_links, dataset1, dataset2)
    # print(len(features))
    
    # Filter pairs based on the given criteria
    index_pairs = []
    
    # Set the threshold of names_not_completely_different function (firstname check after initial comparison)
    threshold = 0.7

    for index, row in features.iterrows():
        a_index = index[0]
        c_index = index[1]
        
        # Bionomia data does not have author abbreviation column to be matched
        # Store the record pairs if their author abbreviations are the same
        # if row['authorAbbrv'] == 1:
        #     index_pairs.append(index)
            
        # Waterfall model to check criteria
        if row['lastName'] >= lastName_threshold:
            if row['firstName'] >= firstName_threshold:
                index_pairs.append(index)
            if row['firstName_initial'] == 1:
                firstName1 = dataset1.at[index[0], 'firstName']
                firstName2 = dataset2.at[index[1], 'firstName']
                if not (is_initial(firstName1) or is_initial(firstName2)) and names_not_completely_different(firstName1, firstName2, threshold):
                    index_pairs.append(index)
            if row['first_name_initial'] == 1:
                if not (is_initial(firstName1) or is_initial(firstName2)) and names_not_completely_different(firstName1, firstName2, threshold):
                    index_pairs.append(index)
            if row['firstName_initial'] == 1 and row['dateOfBirth'] == 1:
                index_pairs.append(index)
            if row['first_name_initial'] == 1 and row['dateOfBirth'] == 1:
                index_pairs.append(index)
            # if row['dateOfBirth'] == 1:
            #     index_pairs.append(index)
            if row['dateOfBirth'] == 1 and row['dateOfDeath'] == 1:
                index_pairs.append(index)
            # Check if b's label is in a's aliases list
            if pd.notna(dataset1.at[a_index, 'aliases']) and row['dateOfBirth'] == 1:
                aliases_list = [alias.strip() for alias in dataset1.at[a_index, 'aliases'].split(',')]
                for alias in aliases_list:
                    if pd.notna(dataset2.at[c_index, 'label']) and pd.notna(alias) and ratio(dataset2.at[c_index, 'label'], alias) >= 0.9:
                        index_pairs.append(index)
                        break              
            # Check if a's label is in b's aliases list
            if pd.notna(dataset2.at[c_index, 'acceptedNames']) and row['dateOfBirth'] == 1:
                aliases_list = [alias.strip() for alias in dataset2.at[c_index, 'acceptedNames'].split(',')]
                for alias in aliases_list:
                    if pd.notna(dataset1.at[a_index, 'label']) and pd.notna(alias) and ratio(dataset1.at[a_index, 'label'], alias) >= 0.9:
                        index_pairs.append(index)
                        break

    # Remove duplicate pairs
    index_pairs = list(set(index_pairs))
    
    # Convert index_pairs to a MultiIndex object
    multi_index = pd.MultiIndex.from_tuples(index_pairs, names=["wikiID", "bioID"])

    return multi_index

In [ ]:
# Load data
a = pd.read_csv('a.csv',chunksize=10000,encoding='utf-8',on_bad_lines='skip',engine='python')
a = pd.concat(a)
a['wikiID'] = a['id']
a = a.set_index('id')

b = pd.read_csv('b.csv',chunksize=10000,encoding='utf-8',on_bad_lines='skip',engine='python')
b = pd.concat(b)
b['harvardIndex'] = b['id']
b = b.set_index('id')

c = pd.read_csv('c.csv',chunksize=10000,encoding='utf-8',on_bad_lines='skip',engine='python')
c = pd.concat(c)
c['acceptedNames'] = remove_spec_in_col(c,'acceptedNames')
c.rename(columns={'Unnamed: 0': 'bioID'}, inplace=True)

# Find out the ground truth matches of wikidata and harvard index using id in the records
a['harvardIndex'] = pd.to_numeric(a['harvardIndex'],errors='coerce') 
temp = pd.merge(a, b, how='inner', on=None, left_on='harvardIndex', right_on='harvardIndex',
                  left_index=False, right_index=False, sort=False,
                  suffixes=('_wiki', '_harvard'), copy=False, indicator=False)

true_matches_WH = define_true_pairs(temp['wikiID'],temp['harvardIndex'].astype(int),'wikiID','harvardIndex')
# print("\nWikiID and havardIndex pairs as true matches:")
# print(true_matches)

# Print out the precentage
print('There are '+ str(len(true_matches_WH)) +' HarvardIndex records in Wikidata, which is ' + str(len(true_matches_WH)/len(a)*100) +'%')
print('There are '+ str(len(true_matches_WH)) +' Wikidata records in HarvardIndex, which is ' + str(len(true_matches_WH)/len(b)*100) +'%')

# Find out the ground truth matches of wikidata and bionomia using id in the records
temp = pd.merge(a, c, how='inner', on=None, left_on='wikiID', right_on='wikidata',
                  left_index=False, right_index=False, sort=False,
                  suffixes=('_wiki', '_bionomia'), copy=False, indicator=False)

true_matches_WB = define_true_pairs(temp['wikiID'],temp['bioID'].astype(int),'wikiID','bioID')
# print("\nWikiID and bionomia pairs as true matches:")
# print(true_matches)

# Print out the precentage
print('There are '+ str(len(true_matches_WB)) +' Bionomia records in Wikidata, which is ' + str(len(true_matches_WB)/len(a)*100) +'%')
print('There are '+ str(len(true_matches_WB)) +' Wikidata records in Bionomia, which is ' + str(len(true_matches_WB)/len(c)*100) +'%')


# Preprocess and clean data
dfa = a.copy()
dfb = b.copy()
dfc = c.copy()

# Set indices
dfa.set_index('wikiID', inplace=True)
dfb.set_index('harvardIndex', inplace=True)
dfc.set_index('bioID', inplace=True)

# Apply the split_full_name function to separate first name and last name
dfa[['first_name', 'last_name']] = dfa['label'].apply(lambda x: pd.Series(split_full_name(x)))
# Apply the convert_to_initial function to the first name column
dfa['first_name_initial'] = dfa['first_name'].apply(convert_to_initial)

# Apply the split_full_name function to separate first name and last name
dfb[['first_name', 'last_name']] = dfb['Standard/Label Name'].apply(lambda x: pd.Series(split_full_name(x)))
# Apply the convert_to_initial function to the first name column
dfb['first_name_initial'] = dfb['first_name'].apply(convert_to_initial)

# Apply the split_full_name function to separate first name and last name
dfc[['first_name', 'last_name']] = dfc['label'].apply(lambda x: pd.Series(split_full_name(x)))
# Apply the convert_to_initial function to the first name column
dfc['first_name_initial'] = dfc['first_name'].apply(convert_to_initial)

# Apply the convert_to_initial function to the first name column in Wiki
dfa['firstName_initial'] = dfa['firstName'].apply(convert_to_initial)
# Apply the convert_to_initial function to the first name column in HI
dfb['firstName_initial'] = dfb['firstName'].apply(convert_to_initial)
# Apply the convert_to_initial function to the first name column in Bionomia
dfc['firstName_initial'] = dfc['firstName'].apply(convert_to_initial)

# Drop duplicated last name column
dfa.drop('last_name', axis=1, inplace=True)
dfb.drop('last_name', axis=1, inplace=True)
dfc.drop('last_name', axis=1, inplace=True)

print('Finished data preparation for Wikidata, HI data and Bionomia data')

In [ ]:
# Finding all possible matches between wikidata and harvard index (all data)
temp_index = generate_match_pairs(dfa, dfb, lastName_threshold=0.9, firstName_threshold=0.85)
# print(len(temp_index))

# Combine the possible matches with the true matches generated by exact id matching
index_pairs_WH = combine_multi_index(temp_index, true_matches_WH)
print("There are {} possible matches found based on the given criteria".format(len(index_pairs_WH)))

# Print out the precentage
print('There are '+ str(len(index_pairs_WH)) +' HarvardIndex records in Wikidata, which is ' + str(len(index_pairs_WH)/len(a)*100) +'%')
print('There are '+ str(len(index_pairs_WH)) +' Wikidata records in HarvardIndex, which is ' + str(len(index_pairs_WH)/len(b)*100) +'%')

# Merge the matches with the original data
matched_dfa = dfa.loc[index_pairs_WH.get_level_values('wikiID')].reset_index()
matched_dfb = dfb.loc[index_pairs_WH.get_level_values('harvardIndex')].reset_index()

# Combine the matched DataFrames side by side
combined_matches_WH = pd.concat([matched_dfa, matched_dfb], axis=1)

# Optionally add a label to identify matched rows
combined_matches_WH['matched'] = True

# Display the combined DataFrame
combined_matches_WH

In [ ]:
# Finding all possible matches between wikidata and harvard index (all data)
temp_index = generate_match_pairs_bio(dfa, dfc, lastName_threshold=0.9, firstName_threshold=0.85)
# print(len(temp_index))

# Combine the possible matches with the true matches generated by exact id matching
index_pairs_WB = combine_multi_index(temp_index, true_matches)
print("There are {} possible matches found based on the given criteria".format(len(index_pairs_WB)))

# Print out the precentage
print('There are '+ str(len(index_pairs_WB)) +' Bionomia records in Wikidata, which is ' + str(len(index_pairs_WB)/len(a)*100) +'%')
print('There are '+ str(len(index_pairs_WB)) +' Wikidata records in Bionomia, which is ' + str(len(index_pairs_WB)/len(c)*100) +'%')

# Merge the matches with the original data
matched_dfa = dfa.loc[index_pairs_WB.get_level_values('wikiID')].reset_index()
matched_dfc = dfc.loc[index_pairs_WB.get_level_values('bioID')].reset_index()

# Combine the matched DataFrames side by side
combined_matches_WB = pd.concat([matched_dfa, matched_dfc], axis=1)

# Optionally add a label to identify matched rows
combined_matches_WB['matched'] = True

# Display the combined DataFrame
combined_matches_WB